In [1]:
import pandas as pd
import numpy as np
import copy
import random
from math import sqrt
import scipy.stats as sts

from tqdm import tqdm_notebook

from supplement_package import game
from supplement_package import variables_pecan

pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
import matplotlib.pyplot as plt

import seaborn as sns
sns.set()

In [3]:
from supplement_package.game.stackelberg import StackelbergPlayer

In [4]:
import gurobipy as gp

from supplement_package.gurobi_implementation.gurobi import GurobiSolution

## Data preliminaries 

In [5]:
agent_keys = [661, 1642, 2335, 2361, 2818, 3039, 3456, 3538, 4031, 4373, 4767, 5746, 6139, 7536, 7719, 7800, 7901, 7951, 8156, 8386, 8565, 9019, 9160, 9922, 9278]

dataframe_dict = dict()
for key in agent_keys:
    dataframe_dict.update({key : pd.read_csv('/Users/ishilov/Documents/risk_paper/risk_paper/data/df_{}.csv'.format(key))})

community_size = len(dataframe_dict)

for key in agent_keys:
    cond_min = (dataframe_dict[key]['demand'].quantile(0.001) <= dataframe_dict[key]['demand'])
    cond_max = (dataframe_dict[key]['demand'] <= dataframe_dict[key]['demand'].quantile(0.999))
    dataframe_dict[key] = dataframe_dict[key][cond_min & cond_max]

In [6]:
def generate_new_params(sample_size, community_size):
    res = {}

    for sample in range(sample_size):
        A_tilde = [random.uniform(0,1) for i in range(community_size)]
        B_tilde = [random.uniform(0,1) for i in range(community_size)]

        a = [random.uniform(0,1) for i in range(community_size)]
        b = [random.uniform(0,1) for i in range(community_size)]
        d = [random.uniform(0,1) for i in range(community_size)]

        #d_target = [[random.uniform(0,8) for j in range(len(probabilities))] for i in range(community_size)]
        #g_res = [[random.uniform(0,3) for j in range(len(probabilities))] for i in range(community_size)]

        #g_res = np.array(g_res)
        #d_target = np.array(d_target)

        risk_aversion = [random.uniform(0,1) for i in range(community_size)]

        res.update({sample : {'A_tilde' : A_tilde,
                        'B_tilde' : B_tilde,
                        'a' : a,
                        'b' : b,
                        'd' : d,
                        'risk_aversion' : risk_aversion}})

    res_reformed = {(i, key) : res[i][key] for i in range(sample_size) for key in res[0].keys()}
    mindx = pd.MultiIndex.from_tuples(res_reformed.keys())
    df = pd.DataFrame(list(res_reformed.values()), index = mindx)
    df.to_csv(f'../data/param_{sample_size}.csv')


In [7]:
def text_to_adj_matrix(matrix_path):
    res = []
    with open(matrix_path) as file:
        for s in file:
            string = ''.join(s.strip().strip(',').split(', '))
            lst_temp = [int(sym) for sym in string]

            res.append(lst_temp)

    return res

In [8]:
#generate_new_params(2000)

In [9]:
def read_df_param(sample_size):
    df_param = pd.read_csv(f'../data/param_{sample_size}.csv')
    df_param.rename({'Unnamed: 0' : 'Sample', 'Unnamed: 1' : 'Parameter'}, axis=1, inplace= True)
    df_param.set_index(['Sample', 'Parameter'], inplace=True)

    return df_param

In [10]:
def distribution_build(sample_size, agent_keys):
    res = {}
    
    for key in agent_keys:
        #chunks_demand = int(demand_dict[key][0].size / sample_size)
        #chunks_generation = int(solar_dict[key][0].size / sample_size)


        #probas_demand = [np.trapz(demand_dict[key][1][i * sample_size : (i + 1) * sample_size],
        #                            demand_dict[key][0][i * sample_size : (i + 1) * sample_size])
        #                            for i in range(chunks_demand)]

        #probas_generation = [np.trapz(generation_dict[key][1][i * sample_size : (i + 1) * sample_size],
        #                            generation_dict[key][0][i * sample_size : (i + 1) * sample_size])
        #                            for i in range(chunks_generation)]

        #res_demand = plt.hist(np.random.choice(demand_dict[key][0], size = sample_size, p = probabilities), bins = int(sample_size / 2))
        #probas_update = res_demand[0] / res_demand[0].sum() if key == 661 else res[661]['probabilities']
        #res.update({key : 
        #            {'values' : res_demand[1], 
        #            'probabilities' : probas_update}})

        hist_demand = plt.hist(dataframe_dict[key]['demand'], bins = sample_size)
        probas_demand, values_demand = hist_demand[0], hist_demand[1]
        probas_demand = probas_demand / probas_demand.sum()

        if 'solar' in dataframe_dict[key].columns:
            hist_solar = plt.hist(dataframe_dict[key][dataframe_dict[key]['solar'] >= 0]['solar'], bins = sample_size)
            probas_solar , values_solar  = hist_solar[0], hist_solar[1]
            probas_solar = probas_demand / probas_demand.sum()

        res.update({key : 
                    {'probas_demand' : probas_demand,
                    'values_demand' : values_demand,
                    'probas_solar' : probas_solar,
                    'values_solar' : values_solar}})

    return res 

In [11]:
def scenario_sampling(sample_size, agent_keys, main_key = 661):
    distribution = distribution_build(sample_size, agent_keys)

    probabilities = distribution[main_key]['probas_demand']

    d_target = []
    g_res = []
    for key in agent_keys:
        d_target.append(distribution[key]['values_demand'][:-1])
        g_res.append(distribution[key]['values_solar'][:-1])

    return probabilities, d_target, g_res

In [12]:
def param_input(df, index = 0):
    A_tilde = list(df.loc[index].loc['A_tilde'])
    B_tilde = list(df.loc[index].loc['B_tilde'])
    a = list(df.loc[index].loc['a'])
    b = list(df.loc[index].loc['b'])
    d = list(df.loc[index].loc['d'])
    risk_aversion = list(df.loc[index].loc['risk_aversion'])

    for i, RA in enumerate(risk_aversion):
        if RA > 0.95:
            risk_aversion[i] = risk_aversion[i] - 0.1

    return A_tilde, B_tilde, a, b, d, risk_aversion

In [13]:
param_amount = 2000

#generate_new_params(param_amount,community_size=community_size)
df_param = read_df_param(param_amount)

In [14]:
def sample_to_csv(scenario_amount, probabilities, d_target, g_res):
    pd.DataFrame(d_target).to_csv(f'../data/df_d_target_{scenario_amount}.csv')
    pd.DataFrame(g_res).to_csv(f'../data/df_g_res_{scenario_amount}.csv')
    pd.DataFrame(probabilities).to_csv(f'../data/df_probabilities_{scenario_amount}.csv')

In [15]:
def sample_from_csv(scenario_amount):
    df_d_target = pd.read_csv(f'../data/df_d_target_{scenario_amount}.csv').drop('Unnamed: 0', axis = 1)
    df_g_res = pd.read_csv(f'../data/df_g_res_{scenario_amount}.csv').drop('Unnamed: 0', axis = 1)
    df_probabilities =pd.read_csv('../data/df_probabilities_100.csv').drop('Unnamed: 0', axis = 1)
    
    probabilities = df_probabilities.values.squeeze()


    d_target = []
    for _, row in df_d_target.iterrows():
        d_target.append(row.values)

    g_res = []
    for _, row in df_g_res.iterrows():
        g_res.append(row.values)

    return probabilities, d_target, g_res
    

In [16]:
probabilities, d_target, g_res = sample_from_csv(100)
#A_tilde, B_tilde, a, b, d, risk_aversion = param_input(df_param)

In [17]:
def agents_list_optimistic_total(A_tilde, B_tilde, a, b, d, risk_aversion, probabilities, connection_matrix, d_target, g_res):
    agents = []
    StackelbergPlayer.community_size = community_size
    StackelbergPlayer.probabilities = probabilities

    epsilon = 0.001
    alpha = [[proba/(1 - min(risk_aversion)) for proba in probabilities] for i in range(community_size)]
    #alpha = [[0.2 for proba in probabilities] for i in range(community_size)]
    gamma = [proba/(1 - min(risk_aversion))for proba in probabilities]

    j_max = [10 for i in range(community_size)]

    for i in range(community_size):
        agent = StackelbergPlayer(i, d_target[i], g_res[i], a[i], b[i], d[i], 
                    A_tilde[i], B_tilde[i], D_min[i], D_max[i], 
                    G_min[i], G_max[i], risk_aversion[i], Kappa[i], Cost[i], connection_matrix[i],
                    probabilities = probabilities,
                    alpha = alpha[i], 
                    gamma = gamma, 
                    insurance_bound=j_max[i])
        
        agents.append(agent)

    return agents

In [18]:
def vars_to_dict(model, list_vars):
    var_names = []
    
    for var in model.getVars():
        var_names.append(var.VarName)

    dict_res = {}
    for name, var in zip(var_names, list_vars):
        dict_res.update({name : var})

    return dict_res 


In [19]:
def gurobi_experiment(df_param, index, err_track, connection_matrix, probabilities, d_target, g_res, solution_type, verbosity = 0):
    A_tilde, B_tilde, a, b, d, risk_aversion = param_input(df_param, index = index)

    if solution_type == 'without_IC':
        agents = agents_list_optimistic_total(A_tilde, B_tilde, a, b, d, risk_aversion, probabilities, connection_matrix, d_target, g_res)

    model_1 = gp.Model()
    setup = GurobiSolution(agents=agents,
                    model = model_1,
                    solution_type=solution_type)

    model_1.setParam('OutputFlag', verbosity)

    setup.build_model()

    try:
        model_1.optimize()

        list_vars = model_1.X
        dict_vars = vars_to_dict(model_1, list_vars)
        objective_val = model_1.getObjective().getValue()
        
        
        
    except:
        err_track.append(index)
        list_vars = ['err']
        objective_val = 'err'
        dict_vars = vars_to_dict(model_1, list_vars)

    return dict_vars, objective_val, model_1

In [20]:
D_min = [0 for i in range(community_size)]
D_max = [dataframe_dict[i].demand.max() for i in dataframe_dict.keys()]

G_min = [0 for i in range(community_size)]
G_max = [dataframe_dict[i].grid.max() for i in dataframe_dict.keys()]

Kappa = [[10 if i!=j else 0 for i in range(community_size)] for j in range(community_size)]

Cost = [[1 for i in range(community_size)] for j in range(community_size)]


In [21]:
connection_matrix_2 = text_to_adj_matrix('../matrices/matrix_2.txt')

In [22]:
def experiment(probabilities, connection_matrix, d_target, g_res,
                df_param, solution_type='without_IC', verbosity = 0):

    total_results= {}
    err_track = []
    for index in tqdm_notebook(df_param.index.levels[0]):
        vars, objective, model = gurobi_experiment(df_param, index, err_track, connection_matrix, probabilities, d_target, g_res, solution_type, verbosity)
        total_results.update({index: {'vars' : vars,
                                    'objective' : objective}})


    results_vars = pd.DataFrame(data = [list(total_results[0]['vars'].values())],
                                        columns=total_results[0]['vars'].keys())

    results_vars['objective'] = total_results[0]['objective']

    for i in tqdm_notebook(range(1, len(total_results))):
        df_temp = pd.DataFrame(data = [list(total_results[i]['vars'].values())],
                                            columns=total_results[i]['vars'].keys())
                                
        df_temp['objective'] = total_results[i]['objective']

        results_vars = pd.concat([results_vars, df_temp], ignore_index=True)

    return total_results, results_vars, model

## Experiment

In [23]:
res_without_IC, df_without_IC, model_without_IC = experiment(probabilities, connection_matrix_2, d_target, g_res,
            df_param, solution_type='without_IC')

/var/folders/98/gq_0lybs5k55th2hc4jstmyw00m7tv/T/ipykernel_30993/1136839945.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for index in tqdm_notebook(df_param.index.levels[0]):


  0%|          | 0/2000 [00:00<?, ?it/s]

Set parameter Username

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2022-04-02


/var/folders/98/gq_0lybs5k55th2hc4jstmyw00m7tv/T/ipykernel_30993/1136839945.py:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(1, len(total_results))):


  0%|          | 0/1999 [00:00<?, ?it/s]

In [24]:
df_without_IC

,D_0_0,D_0_1,D_0_2,D_0_3,D_0_4,D_0_5,D_0_6,D_0_7,D_0_8,D_0_9,...,u_24_91,u_24_92,u_24_93,u_24_94,u_24_95,u_24_96,u_24_97,u_24_98,u_24_99,objective
0,2.734070,2.749192,2.764204,2.779123,2.793966,2.808727,2.823447,2.838147,2.852847,2.867570,...,73.825819,74.047618,74.269049,74.534429,74.800226,75.060014,75.312898,75.566299,75.810472,0.0
1,3.161151,3.187446,3.213755,3.240016,3.266278,3.292518,3.318839,3.345183,3.371504,3.397591,...,94.170966,93.156756,93.474046,93.776198,94.153497,94.545998,94.945545,95.345746,95.753251,0.0
2,3.145341,3.161979,3.178635,3.195315,3.212023,3.228761,3.245536,3.262350,3.279212,3.296120,...,86.046270,86.240560,86.440000,86.646369,83.106481,83.366756,83.633795,82.791268,83.068112,0.0
3,3.347357,3.372097,3.396927,3.421790,3.446693,3.471643,3.496636,3.521693,3.546824,3.572030,...,103.026267,103.239993,103.474162,103.715398,102.784379,103.033742,102.091035,102.346663,99.485502,0.0
4,2.875456,2.886972,2.898502,2.910011,2.921560,2.934053,2.947758,2.961552,2.975403,2.989360,...,48.453035,48.464557,47.830663,47.843479,47.856384,47.869371,47.882213,47.625633,47.637952,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,3.092264,3.106580,3.120918,3.135287,3.149695,3.164153,3.178659,3.193233,3.207887,3.222631,...,103.927701,102.824424,103.157369,103.513375,103.883579,102.452110,102.840850,103.238654,103.637944,0.0
1996,2.976509,2.988560,3.001866,3.015891,3.029884,3.043882,3.057943,3.072114,3.086421,3.100912,...,101.888780,102.352745,102.823778,101.844977,102.312318,102.779559,103.247231,103.713734,104.178706,0.0
1997,3.086795,3.105853,3.124933,3.143993,3.163072,3.182170,3.201293,3.220441,3.239626,3.258849,...,100.424805,100.552523,100.683945,100.819283,100.955679,101.075411,99.821148,99.960292,100.123248,0.0
1998,2.770147,2.782242,2.794246,2.806188,2.818098,2.830010,2.841925,2.853712,2.865447,2.877305,...,89.527454,89.756532,89.988362,90.222214,90.458025,90.694441,86.675669,86.908484,87.135325,0.0


In [25]:
df_without_IC.to_csv(f'../data/without_IC_results_{len(df_without_IC)}.csv')

In [26]:
def agent_utility_from_df_row(df_row, agents):
    res = []
    for agent in agents:
        eta_idx = f'eta_{agent.id}'
        agent_obj = df_row[eta_idx]

        for proba in agent.probabilities_ind:
            u_idx = f'u_{agent.id}_{proba}'
            w_idx = f'W_{agent.id}_{proba}'
            j_idx = f'J_{agent.id}_{proba}'

            agent_obj += (agent.alpha[proba] * df_row[j_idx]
                        + agent.gamma[proba] * df_row[w_idx]
                        + agent.probabilities[proba] / (1 - agent.risk_aversion) * df_row[u_idx])

        res.append(agent_obj)

    return res

In [27]:
def IC_utility_from_df_row(df_row, agents):
    res = 0

    for agent in agents:
        for proba in agent.probabilities_ind:
            j_idx = f'J_{agent.id}_{proba}'

            res += - agent.alpha[proba] * df_row[j_idx] + agent.probabilities[proba] * df_row[j_idx]

    return res

In [28]:
agent_objectives = []
for index, row in df_without_IC.iterrows():
    A_tilde, B_tilde, a, b, d, risk_aversion = param_input(df_param, index = index)
    agents = agents_list_optimistic_total(A_tilde, B_tilde, a, b, d, risk_aversion, probabilities, connection_matrix_2, d_target, g_res)
    agent_objectives.append(agent_utility_from_df_row(row, agents))

In [29]:
dict_df_param = df_param.to_dict()
for i in range(community_size):
    for trial_idx, trial in enumerate(agent_objectives):
        dict_df_param[f'{i}'].update({(trial_idx, 'objective') : trial[i]})

df_param = pd.DataFrame(dict_df_param).sort_index(level=0)
df_param.to_csv(f'../data/without_IC_params_{2000}_with_obj.csv')

In [30]:
df_without_IC.loc[0]['W_8_0']

-9.401123621806084

In [31]:
agent_objectives

[[26.430635110831812,
  32.53706764872962,
  4479.276347052134,
  104.98763858811706,
  41.32970198654181,
  37.8212908013025,
  72.73680053226755,
  58.75292465563517,
  14.77479295406732,
  45.54743046602139,
  24.594033763653766,
  62.918991736283736,
  22.006704444394828,
  64.62207360303226,
  26.537808469352573,
  16.770568045982934,
  74.78900825933546,
  654.5269611576902,
  27.546053118096417,
  1131.342770017828,
  36.83125538840502,
  113.8521000980189,
  73.55944145963987,
  22.370856982264627,
  269.4160490321212],
 [151.40736868023998,
  164.25660833834556,
  213.92928120218167,
  572.2019746559453,
  127.81679193324786,
  187.76881816886635,
  43.576638011592614,
  38.16783923264337,
  61.06496369687681,
  171.6954924030331,
  1130.3932514434173,
  61.20081134817467,
  1505.5565416518932,
  170.6372614360657,
  139.6443203130347,
  15.855704301667243,
  384.69132784478205,
  226.9364101249152,
  267.0323707945817,
  49.681679133908375,
  63.029345947001595,
  730.5476127

In [37]:
RA = pd.Series(param_input(df_param)[-1]) 

In [39]:
RA[RA > 0.95] = RA[RA > 0.95] - 0.1

In [40]:
RA

0     0.158294
1     0.244178
2     0.890008
3     0.639790
4     0.415878
5     0.304654
6     0.599623
7     0.534107
8     0.050979
9     0.447363
10    0.163835
11    0.549979
12    0.131348
13    0.528416
14    0.205595
15    0.064832
16    0.540199
17    0.846338
18    0.159428
19    0.863805
20    0.310902
21    0.710784
22    0.593612
23    0.117117
24    0.835381
dtype: float64